# [expt_08]: Filter Image Thresholding and Object Matching Evaluation By Mathematical Methods

In [37]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv

from sklearn.metrics.pairwise import cosine_similarity

In [38]:
img = cv.imread('../dataset/frames/train/video29/00029_1640_flip.jpg',0)
_, img_thresh = cv.threshold(img, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
img_ground_truth = cv.imread('../experiments/img/08/ground_truth_image.jpg',0)
_, img_ground_truth = cv.threshold(img_ground_truth, 100, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

cv.imshow('img', img_thresh)

cv.waitKey(0)
cv.destroyAllWindows()
np.unique(img_ground_truth)

array([  0, 255], dtype=uint8)

In [39]:
img_med = np.median(img_thresh)
img_med_lower = int(max(0, 0.7 * img_med))
img_med_upper = int(min(255, 1.3 * img_med))

img_canny = cv.Canny(img_thresh, img_med_lower, img_med_upper)

sobel_x = cv.Sobel(img_thresh, cv.CV_64F, 1, 0, ksize=3)
sobel_y = cv.Sobel(img_thresh, cv.CV_64F, 0, 1, ksize=3)
img_sobel = cv.magnitude(sobel_x, sobel_y)

img_hanny = cv.Laplacian(cv.GaussianBlur(img_thresh, (3, 3), 0), cv.CV_64F)

img_gaussian = cv.GaussianBlur(img_thresh, (3,3), 0)

cv.imshow('img_canny',img_canny)
cv.imshow('img_sobel',img_sobel)
cv.imshow('img_hanny',img_hanny)
cv.imshow('img_gaussian',img_gaussian)

cv.waitKey(0)
cv.destroyAllWindows()

# correlation coeficient

In [40]:
corrcof_canny = np.corrcoef(img_canny.flatten(), img_ground_truth.flatten())[0,1]
corrcof_sobel = np.corrcoef(img_sobel.flatten(), img_ground_truth.flatten())[0,1]
corrcof_hanny = np.corrcoef(img_hanny.flatten(), img_ground_truth.flatten())[0,1]
corrcof_gaussian = np.corrcoef(img_gaussian.flatten(), img_ground_truth.flatten())[0,1]

print(corrcof_canny,corrcof_sobel,corrcof_hanny,corrcof_gaussian)

0.0604780128403794 0.10917695743374259 -0.0009507603683869323 0.010914765842584299


# distance eculodian

In [41]:
img_canny_diff = np.sqrt((img_canny - img_ground_truth)**2)
img_sobel_diff = np.sqrt((img_sobel - img_ground_truth)**2)
img_hanny_diff = np.sqrt((img_hanny - img_ground_truth)**2)
img_gaussian_diff = np.sqrt((img_gaussian - img_ground_truth)**2)

dis_ecl_canny = np.sum(img_canny_diff)
dis_ecl_sobel = np.sum(img_sobel_diff)
dis_ecl_hanny = np.sum(img_hanny_diff)
dis_ecl_gaussian = np.sum(img_gaussian_diff)

dis_ecl_canny = np.floor(dis_ecl_canny)
dis_ecl_sobel = np.floor(dis_ecl_sobel)
dis_ecl_hanny = np.floor(dis_ecl_hanny)
dis_ecl_gaussian = np.floor(dis_ecl_gaussian)

print(dis_ecl_canny,dis_ecl_sobel,dis_ecl_hanny,dis_ecl_gaussian)

17040.0 16552493.0 4313263.0 inf


c:\Users\shakr\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


# cos theta

In [42]:
img_canny_pixels = img_canny.flatten().reshape(1,-1)
img_sobel_pixels = img_sobel.flatten().reshape(1,-1)
img_hanny_pixels = img_hanny.flatten().reshape(1,-1)
img_gaussian_pixels = img_gaussian.flatten().reshape(1,-1)
img_ground_truth_pixels = img_ground_truth.flatten().reshape(1,-1)

canny_cos_scr = cosine_similarity(img_canny_pixels, img_ground_truth_pixels)[0][0]
sobel_cos_scr = cosine_similarity(img_sobel_pixels, img_ground_truth_pixels)[0][0]
hanny_cos_scr = cosine_similarity(img_hanny_pixels, img_ground_truth_pixels)[0][0]
gaussian_cos_scr = cosine_similarity(img_gaussian_pixels, img_ground_truth_pixels)[0][0]

print(canny_cos_scr,sobel_cos_scr,hanny_cos_scr,gaussian_cos_scr)

0.08129497632150105 0.13726460128942114 -0.0009572710663644648 0.06529305966179212


In [43]:

# Perform template matching
result = cv.matchTemplate(img_canny, img_ground_truth, cv.TM_CCOEFF_NORMED)

# Define a threshold to consider a match
threshold = 0.01
locations = np.where(result >= threshold)

# Draw bounding boxes around the detected objects
for pt in zip(*locations[::-1]):
    print(1)
    bottom_right = (pt[0] + img_ground_truth.shape[1], pt[1] + img_ground_truth.shape[0])
    cv.rectangle(img_sobel, pt, bottom_right, 255, 2)

# print(result)
# # Display the result
cv.imshow('Detected Objects', img_sobel)
cv.waitKey(0)
cv.destroyAllWindows()

1
